In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
train_data = pd.read_csv('news_train.csv')
test_data = pd.read_csv('news_test.csv')

In [4]:
pro = train_data[train_data['뉴스제목'].str.contains('프로배구')].index
pro2 = train_data[train_data['뉴스제목'].str.contains('배구')].index
pro3 = train_data[train_data['뉴스제목'].str.contains('선수')].index
train_data.drop(pro, errors='ignore', inplace=True)

train_data.drop(pro2, errors='ignore', inplace=True)

train_data.drop(pro3, errors='ignore', inplace=True)

In [5]:
train_data.reset_index(drop=True, inplace=True)
train_data

,뉴스제목,주가변동
0,"조원태 회장, 한진 경영권 방어 성공",0
1,지금까지 이런 비례대표 소동은 없었다…각 당 최종 명단은,0
2,‘코로나 위기’ 대한항공은?…조원태 ‘첩첩산중’ 시험대,0
3,"한진 경영권 분쟁, 조원태 승리…불씨는 여전",0
4,"전국 고용센터 가보니…“당장은 생계 잇지만, 재취업 못할까 걱정” [‘코로나19’ ...",0
...,...,...
5343,"올림픽 기억 가물가물해도, 99년생 삼총사 ‘초롱초롱’",1
5344,"대한항공, 코로나19 확진자 8명 발생…새 시즌 준비 차질",1
5345,"‘허수봉 33점’ 한국, 호주 물리치고 4강행 [2022발리볼챌린저컵]",1
5346,"찾았다 토종 라이트, ‘허수봉’",1


In [6]:
train_data['뉴스제목'] = train_data['뉴스제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['뉴스제목'] = test_data['뉴스제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

<ipython-input-6-bd70b415daa6>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['뉴스제목'] = train_data['뉴스제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-6-bd70b415daa6>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['뉴스제목'] = test_data['뉴스제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [7]:
stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','일','년','월','로','위','억','등','전','서']

In [8]:
okt = Okt()

In [9]:
X_train = []
X_test = []

for sentence in tqdm(train_data['뉴스제목']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stop_words] # 불용어 제거
    X_train.append(stopwords_removed_sentence)
for sentence in tqdm(test_data['뉴스제목']):
    tokenized_sentence = okt.morphs(sentence, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stop_words]
    X_test.append(stopwords_removed_sentence)

100%|██████████| 1430/1430 [00:03<00:00, 374.89it/s]


In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)

{'대한항공': 1, '항공': 2, '코로나': 3, '아시아나': 4, '한진': 5, '운항': 6, '종합': 7, '연승': 8, '항공사': 9, '감독': 10, '업계': 11, '인천': 12, '산업': 13, '공항': 14, '재개': 15, '한국': 16, '현대': 17, '신': 18, '노선': 19, '기업': 20, '그룹': 21, '칼': 22, '첫': 23, '길': 24, '대': 25, '명': 26, '점': 27, '하늘': 28, '부터': 29, '주': 30, '만': 31, '탁구': 32, '삼성': 33, '화물': 34, '경제': 35, '없다': 36, '지원': 37, '되다': 38, '매각': 39, '리그': 40, '우리카드': 41, '오늘': 42, '종목': 43, '까지': 44, '을': 45, '꺾다': 46, '인천공항': 47, '기': 48, '지': 49, '신문': 50, '여행': 51, '장': 52, '사업': 53, '캐피탈': 54, '코스피': 55, '만에': 56, '기사': 57, '케이타': 58, '제주': 59, '회장': 60, '순': 61, '항공기': 62, '보다': 63, '성': 64, '에너지': 65, '석': 66, '주요': 67, '화': 68, '보험': 69, '금융': 70, '백신': 71, '뉴스': 72, '통합': 73, '조': 74, '손해': 75, '마감': 76, '송현동': 77, '조원태': 78, '포토': 79, '더': 80, '분기': 81, '고': 82, '추진': 83, '원': 84, '에서': 85, '유빈': 86, '최고': 87, '다시': 88, '다': 89, '적': 90, '정부': 91, '정지': 92, '한국전력': 93, '상위': 94, '화재': 95, '확정': 96, '진에어': 97, '인수': 98, '드론': 99, '재': 100, '제': 101, '

In [11]:
threshold = 2
total_cnt = len(tokenizer.word_index) 
rare_cnt = 0 
total_freq = 0 
rare_freq = 0 

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value
      
vocab_size = total_cnt - rare_cnt + 1

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 7632
등장 빈도가 1번 이하인 희귀 단어의 수: 3414
단어 집합에서 희귀 단어의 비율: 44.73270440251572
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 6.95103328921918


In [12]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [13]:
y_train = np.array(train_data['주가변동'])
y_test = np.array(test_data['주가변동'])

In [14]:
max_len = max(len(i) for i in X_train)
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [15]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/15
67/67 [==============================] - ETA: 0s - loss: 0.6516 - acc: 0.6540
Epoch 1: val_acc improved from -inf to 0.00000, saving model to best_model.h5
67/67 [==============================] - 12s 120ms/step - loss: 0.6516 - acc: 0.6540 - val_loss: 0.9782 - val_acc: 0.0000e+00
Epoch 2/15
67/67 [==============================] - ETA: 0s - loss: 0.5329 - acc: 0.7270
Epoch 2: val_acc improved from 0.00000 to 0.12523, saving model to best_model.h5
67/67 [==============================] - 7s 97ms/step - loss: 0.5329 - acc: 0.7270 - val_loss: 1.5360 - val_acc: 0.1252
Epoch 3/15
67/67 [==============================] - ETA: 0s - loss: 0.4081 - acc: 0.8123
Epoch 3: val_acc improved from 0.12523 to 0.14953, saving model to best_model.h5
67/67 [==============================] - 6s 92ms/step - loss: 0.4081 - acc: 0.8123 - val_loss: 1.8510 - val_acc: 0.1495
Epoch 4/15
67/67 [==============================] - ETA: 0s - loss: 0.3392 - acc: 0.8499
Epoch 4: val_acc improved from 0.14953

In [20]:
loaded_model = load_model('best_model.h5')
loaded_model.evaluate(X_test, y_test)[1]

45/45 [==============================] - 1s 21ms/step - loss: 0.9785 - acc: 0.5755


0.5755244493484497

In [18]:
def sentiment_predict(new_sentence):
  sentence = new_sentence
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True)
  new_sentence = [word for word in new_sentence if not word in stop_words]
  encoded = tokenizer.texts_to_sequences([new_sentence]) 
  pad_new = pad_sequences(encoded, maxlen = max_len) 
  
  score = float(loaded_model.predict(pad_new)) 
  if(score > 0.5):
    print(sentence +"\n {:.2f}% 확률로 상승합니다.\n".format(score * 100))
  else:
    print(sentence +"\n {:.2f}% 확률로 하락합니다.\n".format((1 - score) * 100))

In [23]:
sentiment_predict('불안한 해외여행...아직도 계속되는 공항 대란')
sentiment_predict('사천공항 항공편 운항 7개월 만에 재개')
sentiment_predict('영국, 대한항공-아시아나 시정안 수용…합병 승인 가능성 커져')
sentiment_predict('100만원 육박 PCR 검사비, 역대 최고 유류할증료 해외여행 기대감 발목')

1/1 [==============================] - 0s 19ms/step
불안한 해외여행...아직도 계속되는 공항 대란
 79.96% 확률로 상승합니다.

1/1 [==============================] - 0s 22ms/step
사천공항 항공편 운항 7개월 만에 재개
 84.04% 확률로 상승합니다.

1/1 [==============================] - 0s 22ms/step
영국, 대한항공-아시아나 시정안 수용…합병 승인 가능성 커져
 97.51% 확률로 하락합니다.

1/1 [==============================] - 0s 20ms/step
100만원 육박 PCR 검사비, 역대 최고 유류할증료 해외여행 기대감 발목
 99.76% 확률로 하락합니다.

